In [4]:
%%capture
! pip install git+https://github.com/openai/swarm.git

In [5]:
from swarm import Swarm, Agent
import json
import os
from datetime import datetime, timedelta
import random


In [6]:
# 設定 OpenAI API Key
os.environ["OPENAI_API_KEY"] = "sk-proj-"


In [7]:

# ====================
# 產品資料庫 (模擬)
# ====================

HP_PRODUCTS = {
    "laptop": {
        "pavilion": {
            "name": "HP Pavilion 15",
            "price": "NT$ 25,000 - 35,000",
            "specs": "Intel Core i5/i7, 8GB-16GB RAM, 512GB SSD",
            "warranty": "1年保固",
            "category": "筆記型電腦"
        },
        "envy": {
            "name": "HP ENVY 13",
            "price": "NT$ 35,000 - 45,000",
            "specs": "Intel Core i7, 16GB RAM, 1TB SSD, 13.3吋觸控螢幕",
            "warranty": "1年保固",
            "category": "筆記型電腦"
        }
    },
    "desktop": {
        "pavilion_desktop": {
            "name": "HP Pavilion Desktop",
            "price": "NT$ 20,000 - 30,000",
            "specs": "Intel Core i5, 8GB RAM, 1TB HDD + 256GB SSD",
            "warranty": "1年保固",
            "category": "桌上型電腦"
        }
    },
    "printer": {
        "deskjet": {
            "name": "HP DeskJet 2720",
            "price": "NT$ 2,500 - 3,500",
            "specs": "彩色噴墨印表機, WiFi, 列印/掃描/影印",
            "warranty": "1年保固",
            "category": "印表機"
        },
        "laserjet": {
            "name": "HP LaserJet Pro M404n",
            "price": "NT$ 8,000 - 12,000",
            "specs": "黑白雷射印表機, 網路功能, 38ppm列印速度",
            "warranty": "1年保固",
            "category": "印表機"
        }
    }
}


In [8]:

# 常見問題資料庫
FAQ_DATABASE = {
    "保固": {
        "question": "HP產品保固期多長？",
        "answer": "HP產品一般提供1年有限保固，部分產品可選購延長保固服務。保固範圍包括硬體故障但不包括人為損壞。"
    },
    "驅動程式": {
        "question": "如何下載HP驅動程式？",
        "answer": "請至HP官方網站 support.hp.com，輸入您的產品型號，選擇對應的作業系統下載最新驅動程式。"
    },
    "維修": {
        "question": "如何申請維修服務？",
        "answer": "可透過HP客服專線、官方網站或授權維修中心申請。請準備產品序號和購買證明。"
    }
}

In [9]:

# ====================
# 功能函數
# ====================

def search_product_info(keyword):
    """搜尋產品資訊"""
    try:
        results = []
        keyword = keyword.lower()

        for category, products in HP_PRODUCTS.items():
            for product_key, product_info in products.items():
                if (keyword in product_info["name"].lower() or
                    keyword in product_info["category"].lower() or
                    keyword in product_key.lower()):
                    results.append(product_info)

        if results:
            return {"status": "success", "products": results}
        else:
            return {"status": "not_found", "message": f"找不到關於 '{keyword}' 的產品資訊"}
    except Exception as e:
        return {"status": "error", "message": f"搜尋產品時發生錯誤: {str(e)}"}


In [10]:

def check_warranty_status(serial_number):
    """查詢保固狀態 (模擬)"""
    try:
        # 模擬查詢保固狀態
        purchase_date = datetime.now() - timedelta(days=random.randint(30, 400))
        warranty_end = purchase_date + timedelta(days=365)
        is_valid = datetime.now() < warranty_end

        return {
            "serial_number": serial_number,
            "purchase_date": purchase_date.strftime("%Y-%m-%d"),
            "warranty_end": warranty_end.strftime("%Y-%m-%d"),
            "is_valid": is_valid,
            "remaining_days": max(0, (warranty_end - datetime.now()).days)
        }
    except Exception as e:
        return {"status": "error", "message": f"查詢保固時發生錯誤: {str(e)}"}

def get_troubleshooting_guide(issue_type):
    """取得故障排除指南"""
    troubleshooting_guides = {
        "列印問題": [
            "1. 檢查印表機是否正確連接電源和電腦",
            "2. 確認墨水或碳粉匣是否充足",
            "3. 檢查紙張是否正確放置",
            "4. 重新啟動印表機和電腦",
            "5. 更新印表機驅動程式"
        ],
        "電腦開機問題": [
            "1. 檢查電源線是否正確連接",
            "2. 按住電源鍵10秒完全關機後重新開機",
            "3. 檢查記憶體是否正確安裝",
            "4. 移除所有外接裝置後重新開機",
            "5. 如問題持續，請聯繫技術支援"
        ],
        "WiFi連線問題": [
            "1. 確認WiFi密碼是否正確",
            "2. 重新啟動路由器和設備",
            "3. 忘記WiFi網路後重新連接",
            "4. 檢查網路卡驅動程式是否需要更新",
            "5. 確認路由器設定是否正確"
        ],
        "藍屏問題": [
            "1. 記錄錯誤代碼",
            "2. 重新啟動電腦",
            "3. 檢查最近安裝的軟體或硬體",
            "4. 執行記憶體診斷工具",
            "5. 如問題持續，請備份資料並聯繫技術支援"
        ]
    }

    if issue_type in troubleshooting_guides:
        return {
            "issue": issue_type,
            "steps": troubleshooting_guides[issue_type]
        }
    else:
        return {
            "message": f"目前沒有 '{issue_type}' 的故障排除指南，建議聯繫技術支援獲得協助。"
        }

def search_faq(query):
    """搜尋常見問題"""
    try:
        query = query.lower()
        results = []

        for key, faq in FAQ_DATABASE.items():
            if (query in faq["question"].lower() or
                query in faq["answer"].lower() or
                query in key.lower()):
                results.append(faq)

        if results:
            return {"status": "success", "faqs": results}
        else:
            return {"status": "not_found", "message": f"找不到關於 '{query}' 的常見問題"}
    except Exception as e:
        return {"status": "error", "message": f"搜尋FAQ時發生錯誤: {str(e)}"}

def create_service_ticket(customer_info, issue_description):
    """建立客服工單 (模擬)"""
    ticket_id = f"HP{datetime.now().strftime('%Y%m%d')}{random.randint(1000, 9999)}"

    return {
        "ticket_id": ticket_id,
        "customer_info": customer_info,
        "issue": issue_description,
        "status": "已建立",
        "created_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "expected_response": "24小時內"
    }


In [11]:

# ====================
# Swarm Agents 定義
# ====================

# 總管理 Agent
general_agent = Agent(
    name="HP客服總管",
    instructions="""你是HP客服中心的總管理員。
    根據客戶的問題類型，將請求轉發給適當的專家：
    - 產品資訊、規格、價格查詢 -> 產品諮詢專家
    - 技術問題、故障排除 -> 技術支援專家
    - 保固查詢、維修申請 -> 售後服務專家
    - 一般問題或FAQ -> 客服專家

    請用親切、專業的語調回應客戶，並確保問題得到適當的處理。""",
    functions=[search_product_info, check_warranty_status, get_troubleshooting_guide, search_faq]
)

# 產品諮詢專家 Agent
product_agent = Agent(
    name="產品諮詢專家",
    instructions="""你是HP產品諮詢專家，專門協助客戶了解HP產品。
    使用 search_product_info 函數來回答：
    - 產品規格和功能
    - 價格資訊
    - 產品比較
    - 購買建議

    請提供詳細且實用的產品資訊，幫助客戶做出最佳選擇。
    記得主動詢問客戶的需求，以便提供更精準的建議。""",
    functions=[search_product_info]
)

# 技術支援專家 Agent
technical_agent = Agent(
    name="技術支援專家",
    instructions="""你是HP技術支援專家，專門解決客戶的技術問題。
    使用 get_troubleshooting_guide 函數來提供：
    - 故障排除步驟
    - 技術問題解決方案
    - 驅動程式和軟體支援

    請提供清楚的逐步指導，並在必要時建議客戶聯繫進階技術支援。
    記得詢問客戶問題的詳細情況，以便提供最適合的解決方案。""",
    functions=[get_troubleshooting_guide, create_service_ticket]
)

# 售後服務專家 Agent
service_agent = Agent(
    name="售後服務專家",
    instructions="""你是HP售後服務專家，專門處理保固和維修相關事務。
    使用相關函數來協助：
    - 保固狀態查詢
    - 維修申請流程
    - 客服工單建立

    請提供準確的保固資訊和維修流程指導。
    如需建立維修申請，請確實收集客戶資訊和問題描述。""",
    functions=[check_warranty_status, create_service_ticket]
)

# 客服專家 Agent
support_agent = Agent(
    name="客服專家",
    instructions="""你是HP客服專家，負責回答一般性問題和常見問題。
    使用 search_faq 函數來提供：
    - 常見問題解答
    - 公司政策說明
    - 聯絡資訊

    請提供準確且有幫助的資訊，並保持友善的服務態度。
    如果無法解決客戶問題，請適時轉介給其他專家。""",
    functions=[search_faq, create_service_ticket]
)


In [12]:

# ====================
# 主程式
# ====================

def main():
    # 創建 Swarm client
    client = Swarm()

    print("🖥️  歡迎使用 HP 客服助手！")
    print("=" * 50)
    print("我可以協助您處理以下問題：")
    print("📱 1. 產品資訊查詢 (筆電、桌機、印表機等)")
    print("🔧 2. 技術支援與故障排除")
    print("🛡️  3. 保固查詢與維修申請")
    print("❓ 4. 常見問題解答")
    print("=" * 50)
    print("請輸入您的問題，或輸入 'exit' 結束對話。")

    context_variables = {}
    current_agent = general_agent

    while True:
        user_input = input("\n💬 您的問題: ")

        if user_input.lower() in ['exit', 'quit', 'bye', '退出', '結束']:
            print("\n謝謝使用 HP 客服助手，祝您有美好的一天！👋")
            break

        try:
            response = client.run(
                agent=current_agent,
                messages=[{"role": "user", "content": user_input}],
                context_variables=context_variables
            )

            print(f"\n🤖 {response.agent.name}: {response.messages[-1]['content']}")
            current_agent = response.agent
            context_variables = response.context_variables

        except Exception as e:
            print(f"\n❌ 抱歉，處理您的請求時發生錯誤: {str(e)}")
            print("請稍後再試，或聯繫我們的客服專線獲得協助。")

if __name__ == "__main__":
    main()

🖥️  歡迎使用 HP 客服助手！
我可以協助您處理以下問題：
📱 1. 產品資訊查詢 (筆電、桌機、印表機等)
🔧 2. 技術支援與故障排除
🛡️  3. 保固查詢與維修申請
❓ 4. 常見問題解答
請輸入您的問題，或輸入 'exit' 結束對話。

💬 您的問題: 我想知道

🤖 HP客服總管: 請問您想知道什麼呢？我們提供產品資訊、技術支援、保固查詢及一般問題的協助。請告訴我您的具體問題或需求，讓我幫您轉接給適當的專家。

💬 您的問題: HP DeskJet 2720

🤖 HP客服總管: 以下是有關 HP DeskJet 2720 的產品資訊：

- **名稱**: HP DeskJet 2720
- **價格**: NT$ 2,500 - 3,500
- **規格**: 彩色噴墨印表機, 支援WiFi, 提供列印/掃描/影印功能
- **保固**: 1年保固
- **類別**: 印表機

如果您有其他問題或需要更多資訊，請隨時告訴我！

💬 您的問題: 如何下載驅動程式

🤖 HP客服總管: 我將這個問題轉發給技術支援專家，稍後會有專家聯絡您提供詳細的驅動程式下載指引。如有其他技術問題，歡迎隨時告訴我們！

💬 您的問題: 如何下載HP驅動程式？

🤖 HP客服總管: 要下載HP驅動程式，請按照以下步驟：

1. 造訪HP的官方網站。
2. 尋找「支持與驅動程式」的選項。
3. 輸入您的產品型號或選擇您的產品類別。
4. 找到「軟體及驅動程式」的選項，然後選擇您的操作系統版本。
5. 您將看到可用的驅動程式列表，選擇您需要的驅動程式，然後點擊下載。
6. 下載完成後，按照指示進行安裝。

如果您有任何問題或需要進一步的幫助，請隨時告訴我！


KeyboardInterrupt: Interrupted by user